In [1]:
all_match_ids = []

In [2]:
import requests
import time

class RiotApi(object):
    def __init__(self, api_key: str, region="americas"):
        self.RIOT_API_KEY = api_key
        self.HEADER = {'X-Riot-Token': self.RIOT_API_KEY}
        self.REGION = region
        self.BASE_URL = ".api.riotgames.com/"

    def get_puuid_by_riot_id(self, game_name: str, tag_line: str) -> dict:
        """Get PUUID by Riot ID (gameName and tagLine)
        @param game_name: Riot ID gameName
        @param tag_line: Riot ID tagLine
        @return: json object of account data
        """
        url = f"https://{self.REGION}{self.BASE_URL}riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for PUUID request. Response: {request.text}")
            return {}

    def get_summoner_by_puuid(self, puuid: str) -> dict:
        """Get Summoner Data by PUUID
        @param puuid: Player Universal Unique Identifier
        @return: json object of summoner data
        """
        self.REGION = "na1"
        url = f"https://{self.REGION}{self.BASE_URL}lol/summoner/v4/summoners/by-puuid/{puuid}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for summoner request. Response: {request.text}")
            return {}
        

    def get_match_ids(self, puuid, count=20):
        self.REGION = "americas"
        match_ids = []
        start = 0
        
        while True:
            url = f"https://{self.REGION}{self.BASE_URL}/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
            request = requests.get(url, headers=self.HEADER)
            if request.status_code == 200:
                batch_ids = request.json()
                if not batch_ids:
                    break
                match_ids.extend(batch_ids)
                start += count
                time.sleep(1.2)  # Adding a delay to avoid rate limits
            else:
                print(f"Error: Received status code {request.status_code} for match ID request. Response: {request.text}")
                break

        return match_ids
        
    def match_details(self, match_id):
        self.REGION = "americas"
        url = f"https://{self.REGION}{self.BASE_URL}/lol/match/v5/matches/{match_id}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for summoner request. Response: {request.text}")
            return {}

# Example usage
API_KEY = 'RGAPI-e7a0cc73-2a12-4b8e-b909-71c950ac8132'  # Replace with your valid API key
#game_name_done = ['white space', 'Sheiden', 'Sushee', 'Afflictive', 'Zven']
#tag_line_done = ['srtty', '0001', 'NA1', 'NA1', 'KEKW1']
game_name = ['Zven', 'Dongdanny', 'wxtonytoppwd']
tag_line = ['KEKW1', 'pwdd', 'NA1']

for i in range(len(game_name)):
    ra = RiotApi(API_KEY)

    # Step 1: Get PUUID by Riot ID
    account_data = ra.get_puuid_by_riot_id(game_name[i], tag_line[i])
    puuid = account_data.get('puuid')
    print(f"{i}.")
    print(f"PUUID for {game_name[i]}#{tag_line[i]}: {puuid}")

    time.sleep(1.2)
    # Step 2: Get Summoner Data by PUUID
    if puuid:
        summoner_data = ra.get_summoner_by_puuid(puuid)
        print(f"Summoner data for : {summoner_data}")
        time.sleep(1.2)

        match_ids = ra.get_match_ids(puuid)
        print(f"Match ids length : {len(match_ids)}, data : {match_ids}")
        time.sleep(1.2)

        for match_id in match_ids:
            time.sleep(0.25)
            details = ra.match_details(match_id)
            if details['info']['queueId'] in {420, 430, 440, 400}:
                all_match_ids.append(match_id)
            time.sleep(1.2)
    else:
        print("PUUID not found, cannot retrieve summoner data.")


In [13]:
print(all_match_ids)
print(len(all_match_ids))

['NA1_5020516914', 'NA1_5020407113', 'NA1_5020373724', 'NA1_5020343197', 'NA1_5020323876', 'NA1_5020310796', 'NA1_5020292476', 'NA1_5019827562', 'NA1_5019626406', 'NA1_5019217363', 'NA1_5019180330', 'NA1_5018959316', 'NA1_5018709260', 'NA1_5018682815', 'NA1_5018649918', 'NA1_5018606049', 'NA1_5018437128', 'NA1_5018402732', 'NA1_5018282975', 'NA1_5017945522', 'NA1_5017632580', 'NA1_5017615728', 'NA1_5017598353', 'NA1_5017587594', 'NA1_5017293789', 'NA1_5017192856', 'NA1_5017152911', 'NA1_5016959471', 'NA1_5013830447', 'NA1_5013731483', 'NA1_5013633945', 'NA1_5013524926', 'NA1_5013506943', 'NA1_5013477585', 'NA1_5013461214', 'NA1_5012915234', 'NA1_5012851645', 'NA1_5012809583', 'NA1_5012796260', 'NA1_5012321671', 'NA1_5012303743', 'NA1_5012147763', 'NA1_5012128910', 'NA1_5012105596', 'NA1_5012092396', 'NA1_5011891064', 'NA1_5011524652', 'NA1_5011433003', 'NA1_5011402713', 'NA1_5011376884', 'NA1_5011351253', 'NA1_5011097104', 'NA1_5010987494', 'NA1_5010867546', 'NA1_5010843357', 'NA1_5010

In [14]:
import pickle
with open('list_of_matches_for_testing.pkl', 'wb') as file:
    pickle.dump(all_match_ids, file)

In [15]:
import pickle
with open('list_of_matches.pkl', 'rb') as file:
    l = pickle.load(file)
print(len(l), l)

1763 ['NA1_5020516914', 'NA1_5020407113', 'NA1_5020373724', 'NA1_5020343197', 'NA1_5020323876', 'NA1_5020310796', 'NA1_5020292476', 'NA1_5019827562', 'NA1_5019626406', 'NA1_5019217363', 'NA1_5019180330', 'NA1_5018959316', 'NA1_5018709260', 'NA1_5018682815', 'NA1_5018649918', 'NA1_5018606049', 'NA1_5018437128', 'NA1_5018402732', 'NA1_5018282975', 'NA1_5017945522', 'NA1_5017632580', 'NA1_5017615728', 'NA1_5017598353', 'NA1_5017587594', 'NA1_5017293789', 'NA1_5017192856', 'NA1_5017152911', 'NA1_5016959471', 'NA1_5013830447', 'NA1_5013731483', 'NA1_5013633945', 'NA1_5013524926', 'NA1_5013506943', 'NA1_5013477585', 'NA1_5013461214', 'NA1_5012915234', 'NA1_5012851645', 'NA1_5012809583', 'NA1_5012796260', 'NA1_5012321671', 'NA1_5012303743', 'NA1_5012147763', 'NA1_5012128910', 'NA1_5012105596', 'NA1_5012092396', 'NA1_5011891064', 'NA1_5011524652', 'NA1_5011433003', 'NA1_5011402713', 'NA1_5011376884', 'NA1_5011351253', 'NA1_5011097104', 'NA1_5010987494', 'NA1_5010867546', 'NA1_5010843357', 'NA1

In [90]:
# retieving all the details like avg kills, asssists of a player.
import requests

class RiotApi(object):
    def __init__(self, api_key: str, region="americas"):
        self.RIOT_API_KEY = api_key
        self.HEADER = {'X-Riot-Token': self.RIOT_API_KEY}
        self.REGION = region

    def get_match_history(self, puuid):
        url = f'https://{self.REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
        response = requests.get(url, headers=self.HEADER)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Received status code {response.status_code} for summoner request. Response: {response.text}")
            return []

    def get_match_data(self, match_id):
        url = f'https://{self.REGION}.api.riotgames.com/lol/match/v5/matches/{match_id}'
        response = requests.get(url, headers=self.HEADER)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Received status code {response.status_code} for summoner request. Response: {response.text}")
            return {}

    def get_player_stats(self, match_data, puuid, champion_to_be_used):
        kills = 0
        deaths = 0
        assists = 0
        total_matches = 0
        champ_lvl = 0
        champ_exp = 0

        for player in match_data['info']['participants']:
            if player['puuid'] == puuid:
                kills += player['kills']
                deaths += player['deaths']
                assists += player['assists']
                total_matches += 1
                if player['championName'] == champion_to_be_used:
                    champ_lvl = player['champLevel']
                    champ_exp = player['champExperience']
        
        return kills, deaths, assists, total_matches, champ_lvl, champ_exp

    def get_summoner_by_puuid(self, puuid, country):
        self.REGION = country
        url = f"https://{self.REGION}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for summoner request. Response: {request.text}")
            return {}
    
    def get_ranked_data(self, summoner_id, country):
        self.REGION = country
        url = f'https://{self.REGION}.api.riotgames.com/lol/league/v4/entries/by-summoner/{summoner_id}'
        response = requests.get(url, headers=self.HEADER)
        if response.status_code == 200:
            ranked_data = response.json()
            if ranked_data:
                return ranked_data[0]['tier'], ranked_data[0]['rank']
        print(f"Error: Received status code {response.status_code} for ranked data request. Response: {response.text}")
        return None, None

    def get_average_stats(self, puuid, champion_to_be_used):
        match_ids = self.get_match_history(puuid)
        total_kills = 0
        total_deaths = 0
        total_assists = 0
        total_matches = 0

        for match_id in match_ids:
            match_data = self.get_match_data(match_id)
            if match_data['info']['queueId'] not in {420, 430, 440, 400}:
                continue

            if match_data:
                country = match_id.split('_')[0]
                kills, deaths, assists, match_count, champ_lvl, champ_exp = self.get_player_stats(match_data, puuid, champion_to_be_used)
                total_kills += kills
                total_deaths += deaths
                total_assists += assists
                total_matches += match_count

        if total_matches > 0:
            average_kills = total_kills / total_matches
            average_deaths = total_deaths / total_matches
            average_assists = total_assists / total_matches
        else:
            average_kills = average_deaths = average_assists = 0
        
        summoner_data = self.get_summoner_by_puuid(puuid, country)
        summoner_id = summoner_data['id']
        tier, rank = self.get_ranked_data(summoner_id, country)

        return average_kills, average_deaths, average_assists, tier, rank, champ_lvl, champ_exp


# Example usage
API_KEY = 'RGAPI-70e702d9-1eb0-41e1-a096-fadff6804f51'
puuid = 'gPRDXczDal0ahDXchGTmtCt2Lj2PtW9qNwEkDuRAGg1nhNJDi4_pdCMminAN9idPpGiS1TzuzELLmg'
riot_api = RiotApi(API_KEY)
champion_to_be_used = 'Amumu'
average_kills, average_deaths, average_assists, tier, rank, champ_lvl, champ_exp = riot_api.get_average_stats(puuid, champion_to_be_used)
print("Average Kills:", average_kills)
print("Average Deaths:", average_deaths)
print("Average Assists:", average_assists)
print("Tier:", tier)
print("Rank:", rank)
print("Level of champion to be used in the prediction match:", champ_lvl)
print("Experience of champion to be used in the prediction match:", champ_exp)

Average Kills: 7.071428571428571
Average Deaths: 3.5714285714285716
Average Assists: 5.714285714285714
Tier: CHALLENGER
Rank: I
Level of champion to be used in the prediction match: 0
Experience of champion to be used in the prediction match: 0


In [ ]:
print(details['metadata'])

{'dataVersion': '2', 'matchId': 'NA1_5018282975', 'participants': ['QtIaehsKS5RCM69jZuoXl28gWqxmNzS4OQmHZviq2WvEv90KxfgYLVN0RKIEmfavRRwWGNan0lEjZw', 'eKO45n0sZd2LJzV-pZ_hH5fCXfE8_TyOeELMOV2ncxmbp5mgvDTsa9sNdo9NdWLVXTD6ksALA-a2Zw', '3hAhq3ojUqs1CEl1lO_hIh183UFBsq2Mj91yY2A_mGMOBJquLP6aJr5OF_uGM3i882BPpOQdWYAMGA', 'YcBBE8z_UgTyyC4iQw9qTEEP-zfZBNxJ9LH4warrPu3y01BNdmSofnTQdgTFPuQfkd2KnnKNWVVRGg', '9cL3xH6y45s4Go-y5YlDSBndPAjD_OLVHnFQFDX9BCWm_5dySBDMa4yExsITaMGEhlzzZJiipj0VFQ', 'k6W_VYtwIMb4zWHm1P5N0UwsfwYN2DI1xpCAnmzknlBNorZy62TqJRkta1WyESDz6FIANz3n2kVrBA', 'gPRDXczDal0ahDXchGTmtCt2Lj2PtW9qNwEkDuRAGg1nhNJDi4_pdCMminAN9idPpGiS1TzuzELLmg', 'KjHlPw8D4ZihwRuheY6JEuUvLeT-_B1bywxS-mInDuaPkA3rlHs8H6Y_Po32eTWZ9itAx1AVRWYdgg', 'uM3VlxxulMCF77yraA8bW_D_xy2df77hwp4G6e78aqOpuMR_kgmfsiQ2yLjTzIkp5yYHjyj3ZlatYg', 'H7_t0LT0F6EmwNpPk_aFy0IN94rBJURM64LMyfGbUrTlFvJevf04To0IZvkUFevusdURCT77vUcfSw']}


In [ ]:
print(details['info'].keys())

dict_keys(['endOfGameResult', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])
[{'bans': [{'championId': 119, 'pickTurn': 1}, {'championId': 111, 'pickTurn': 2}, {'championId': 74, 'pickTurn': 3}, {'championId': 157, 'pickTurn': 4}, {'championId': 18, 'pickTurn': 5}], 'objectives': {'baron': {'first': True, 'kills': 1}, 'champion': {'first': False, 'kills': 38}, 'dragon': {'first': True, 'kills': 2}, 'horde': {'first': False, 'kills': 4}, 'inhibitor': {'first': False, 'kills': 0}, 'riftHerald': {'first': True, 'kills': 1}, 'tower': {'first': False, 'kills': 5}}, 'teamId': 100, 'win': True}, {'bans': [{'championId': 163, 'pickTurn': 6}, {'championId': 497, 'pickTurn': 7}, {'championId': 26, 'pickTurn': 8}, {'championId': 18, 'pickTurn': 9}, {'championId': 72, 'pickTurn': 10}], 'objectives': {'baron': {'first': False, 'kills':

In [ ]:
data = {}
team_blue = []
team_red = []

# Loop through participants to separate them into blue and red teams
for i in range(10):
    if details['info']['participants'][i]['teamId'] == 100:
        team_blue.append(i)
    else:
        team_red.append(i)

if details['info']['participants'][team_blue[0]]['win'] == True:
    data['blue_win'] = 1
else: data['blue_win'] = 0

# Initialize variables for blue and red teams
blue_k = 0
blue_a = 0
blue_d = 0
red_k = 0
red_a = 0
red_d = 0

# Calculate stats for blue team
for i in team_blue:
    blue_k += details['info']['participants'][i]['kills']
    blue_d += details['info']['participants'][i]['deaths']
    blue_a += details['info']['participants'][i]['assists']

# Calculate stats for red team
for i in team_red:
    red_k += details['info']['participants'][i]['kills']
    red_d += details['info']['participants'][i]['deaths']
    red_a += details['info']['participants'][i]['assists']

# Calculate KDA (Kills, Deaths, Assists) for each team
blue_kda = (blue_k + blue_a) / max(1, blue_d)  # Avoid division by zero
red_kda = (red_k + red_a) / max(1, red_d)  # Avoid division by zero

# Store data into the dictionary
data['blue_kills'] = blue_k
data['blue_deaths'] = blue_d
data['blue_assists'] = blue_a
data['blue_kda'] = blue_kda

data['red_kills'] = red_k
data['red_deaths'] = red_d
data['red_assists'] = red_a
data['red_kda'] = red_kda
print(data)

{'blue_win': 1, 'blue_kills': 38, 'blue_deaths': 15, 'blue_assists': 54, 'blue_kda': 6.133333333333334, 'red_kills': 15, 'red_deaths': 38, 'red_assists': 16, 'red_kda': 0.8157894736842105}


In [ ]:
import requests

def get_champion_data(api_key):
    url = f"https://ddragon.leagueoflegends.com/cdn/12.8.1/data/en_US/champion.json"
    response = requests.get(url)
    return response.json()

# Example usage
api_key = 'RGAPI-73de7a1d-bd48-401f-9ea2-12e33bd637f2'  # Replace with your valid API key
champion_data = get_champion_data(api_key)

# Create a dictionary mapping champion IDs to their names and classes
champion_classes = {
    "Tank": [],
    "Mage": [],
    "Assassin": [],
    "Marksman": [],
    "Support": [],
    "Fighter": []
}

for champ in champion_data['data'].values():
    # Assume champ['tags'] contains the class information
    for role in champ['tags']:
        if role in champion_classes:
            champion_classes[role].append(champ['name'])

print(champion_classes)
print(champion_classes.keys())

{'Tank': ['Aatrox', 'Alistar', 'Amumu', 'Blitzcrank', 'Braum', 'Camille', "Cho'Gath", 'Darius', 'Dr. Mundo', 'Galio', 'Garen', 'Gnar', 'Hecarim', 'Illaoi', 'Jarvan IV', 'Kled', 'Leona', 'Malphite', 'Maokai', 'Wukong', 'Nasus', 'Nautilus', 'Nunu & Willump', 'Olaf', 'Ornn', 'Poppy', 'Rammus', 'Rell', 'Renekton', 'Sejuani', 'Sett', 'Shen', 'Shyvana', 'Singed', 'Sion', 'Skarner', 'Tahm Kench', 'Trundle', 'Udyr', 'Urgot', 'Volibear', 'Warwick', 'Yorick', 'Zac'], 'Mage': ['Ahri', 'Amumu', 'Anivia', 'Annie', 'Aurelion Sol', 'Azir', 'Bard', 'Brand', 'Cassiopeia', "Cho'Gath", 'Diana', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Galio', 'Gragas', 'Heimerdinger', 'Ivern', 'Janna', 'Jhin', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kennen', "Kog'Maw", 'LeBlanc', 'Lillia', 'Lissandra', 'Lulu', 'Lux', 'Malzahar', 'Maokai', 'Morgana', 'Nami', 'Neeko', 'Nidalee', 'Orianna', 'Renata Glasc', 'Rumble', 'Ryze', 'Seraphine', 'Sona', 'Soraka', 'Swain', 'Sylas', 'Syndra', 'Taliyah', 'Twisted Fate', 'Var

In [ ]:
for key in champion_classes:
    data[f'blue_{key}'] = 0
    data[f'red_{key}'] = 0

for i in team_blue:
    champion_name = details['info']['participants'][i]['championName']
    for key in champion_classes:
        if champion_name in champion_classes[key]:
            champ_lvl = details['info']['participants'][i]['champLevel']
            champ_exp = details['info']['participants'][i]['champExperience']
            data[f'blue_{key}'] += champ_lvl * champ_exp

for i in team_red:
    champion_name = details['info']['participants'][i]['championName']
    for key in champion_classes:
        if champion_name in champion_classes[key]:
            champ_lvl = details['info']['participants'][i]['champLevel']
            champ_exp = details['info']['participants'][i]['champExperience']
            data[f'red_{key}'] += champ_lvl * champ_exp
print(data)

{'blue_win': 1, 'blue_kills': 38, 'blue_deaths': 15, 'blue_assists': 54, 'blue_kda': 6.133333333333334, 'red_kills': 15, 'red_deaths': 38, 'red_assists': 16, 'red_kda': 0.8157894736842105, 'blue_Tank': 92235, 'red_Tank': 0, 'blue_Mage': 160300, 'red_Mage': 104952, 'blue_Assassin': 351162, 'red_Assassin': 175800, 'blue_Marksman': 340354, 'red_Marksman': 114552, 'blue_Support': 115368, 'red_Support': 61920, 'blue_Fighter': 378711, 'red_Fighter': 113880}


In [ ]:
def filter_5v5_matches(match_details):
    # Define 5v5 queue IDs for Summoner's Rift
    summoners_rift_5v5_queue_ids = {420, 430, 440, 400}
    
    # Filter matches
    filtered_matches = [match for match in match_details if match['queueId'] in summoners_rift_5v5_queue_ids]
    
    return filtered_matches

# Example usage:
all_match_data = [...]  # Assume this is your list of match data
filtered_matches = filter_5v5_matches(all_match_data)


In [ ]:
'baronKills': participant['baronKills'],
 'dragonKills': participant['dragonKills']